In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../src')
from ecomplexity import ecomplexity

# 小数点以下 桁数 6
pd.options.display.float_format = '{:.6f}'.format


In [7]:
import initial_condition
from analysis import weight


In [3]:
global data_dir, output_dir
data_dir = '../../data/interim/internal/filtered_before_agg/'
output_dir = '../../data/interim/internal/filtered_after_agg/'


In [6]:
# 初期条件
ar = initial_condition.AR
year_style = initial_condition.YEAR_STYLE

year_start = initial_condition.YEAR_START
year_end = initial_condition.YEAR_END
year_range = initial_condition.YEAR_RANGE

classification = initial_condition.CLASSIFICATION
class_weight = initial_condition.CLASS_WEIGHT
applicant_weight = initial_condition.APPLICANT_WEIGHT


In [26]:
# 全体
all_df = pd.read_csv(f'{data_dir}addedclassification.csv', 
                     encoding='utf-8', 
                     sep=',', 
                     usecols=['reg_num', 
                              'right_person_name', 
                              f'{ar}_{year_style}', 
                              f'{classification}'], 
                     dtype={'reg_num': str, 
                            'right_person_name': str, 
                            f'{ar}_{year_style}': np.int64, 
                            f'{classification}': str})

all_df = all_df[all_df[f'{ar}_{year_style}'].isin(range(year_start, year_end+1))]\
               .drop_duplicates()\
# display(all_df.head())


# 各期間
sep_year_df_dict = {}

for year in range(year_start, year_end+1, year_range):
    sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_year'].isin(range(year, year+year_range))]


In [21]:
# 上位何%までを抽出するか
def extract_top(reg_num_df: pd.DataFrame, 
                top_p_or_num: tuple):
    if top_p_or_num[0] == 'p': top = (reg_num_df['right_person_name'].nunique()*top_p_or_num[1])//100
    elif top_p_or_num[0] == 'num': top = top_p_or_num[1]
    top_right_person_list = reg_num_df.copy().groupby(['right_person_name'])[['reg_num']].sum()\
                                               .sort_values(['reg_num'], ascending=False)\
                                               .reset_index(drop=False)\
                                               .head(top_p_or_num)\
                                               ['right_person_name'].to_list()
    
    reg_num_top_p_df = reg_num_df[reg_num_df['right_person_name'].isin(top_right_person_list)].copy()
    # reg_num_top_p_df['segment'] = reg_num_top_p_df['period'].str[:4].astype(np.int64)
    return reg_num_top_p_df
    # sep_year_top_p_df_dict[p][period] = sep_year_top_p_df.copy()
    

In [13]:
# 全体
if class_weight == 'fraction':
    all_reg_num_df = weight.by_classification(all_df)
else:
    all_reg_num_df = all_df.groupby(['right_person_name', 'ipc_class'])[['reg_num']].nunique().reset_index(drop=False)
all_reg_num_df['segment'] = f'{year_start}-{year_end}'


# 期間ごと
sep_year_reg_num_df_dict = {}
sep_year_reg_num_top_p_df_dict = {}
for period, sep_year_df in sep_year_df_dict.items():
    if weight == 'fraction':
        sep_year_reg_num_df_dict[period] = weight_by_ipc(sep_year_df)
    else:
        all_reg_num_df = sep_year_df.groupby(['right_person_name', 'ipc_class'])[['reg_num']].nunique().reset_index(drop=False)
    sep_year_reg_num_df_dict[period]['segment'] = period


In [17]:
pd.concat([all_reg_num_df] + list(sep_year_reg_num_df_dict.values()), axis='index')

,right_person_name,ipc_class,reg_num,segment
0,株式会社リコー,G03G,13970.000000,1981-2010
1,キヤノン株式会社,G03G,13956.500000,1981-2010
2,富士通株式会社,G06F,12997.000000,1981-2010
3,キヤノン株式会社,H04N,12147.500000,1981-2010
4,日本電気株式会社,G06F,11593.500000,1981-2010
...,...,...,...,...
152533,株式会社ＧＳユアサ,F21Y,0.330000,2001-2010
152534,アルパイン株式会社,G03B,0.330000,2001-2010
152535,アプライドリサーチ株式会社,H04N,0.330000,2001-2010
152536,アプライドリサーチ株式会社,G01L,0.330000,2001-2010


In [ ]:
if 
all_reg_num_top_p_df = extract_top(all_reg_num_df, 
                                   top_p_or_num)

display(all_reg_num_top_p_df.head())
sep_year_reg_num_top_p_df_dict[period] = extract_top(sep_year_reg_num_df_dict[period], top_p)

sep_year_reg_num_top_p_df = pd.concat(sep_year_reg_num_top_p_df_dict.values(), 
                                      axis='index')

display(sep_year_reg_num_top_p_df)


In [ ]:
print('特許数（次数削減前）:', all_reg_num_top_p_df.groupby('right_person_name')['reg_num'].sum().reset_index(drop=False)['reg_num'].sum())
print('特許権者（次数削減前）:', all_reg_num_top_p_df['right_person_name'].nunique())
print('IPCクラス（次数削減前）:', all_reg_num_top_p_df['ipc_class'].nunique())
display(all_reg_num_top_p_df.head())


# 各期間
# sep_year_df_dict = {}

# for year in range(year_start, year_end+1, year_range):
#     sep_year_df_dict[f'{year}-{year+year_range-1}'] = all_df[all_df[f'{ar}_year'].isin(range(year, year+year_range))]
#     print(f'=============={year}-{year+year_range-1}==============')
#     print('特許数（次数削減前）:', sep_year_df_dict[f'{year}-{year+year_range-1}']['reg_num'].nunique())
#     print('特許権者（次数削減前）:', sep_year_df_dict[f'{year}-{year+year_range-1}']['right_person_name'].nunique())
#     print('IPCクラス（次数削減前）:', sep_year_df_dict[f'{year}-{year+year_range-1}']['ipc_class'].nunique())
#     print('=====================================\n')


In [ ]:
def add_schmoch(df: pd.DataFrame):
    